In [1]:

from scipy.spatial import distance as dist
import numpy as np
import time
import dlib
import cv2
from collections import OrderedDict

# definir dos constantes, una para la proporcion de aspecto del ojo para indicar
# parpadeo y luego una segunda constante para el número de
# marcos el ojo debe estar por debajo del threshold
EYE_AR_THRESH = 0.23
EYE_AR_CONSEC_FRAMES = 35

# inicializar los contadores de fotogramas y el número total de parpadeos
COUNTER = 0

# definir un diccionario que mapee los índices de la cara
# puntos de referencia a regiones específicas de la cara
FACIAL_LANDMARKS_IDXS = OrderedDict([
    ("mouth", (48, 68)),
    ("right_eyebrow", (17, 22)),
    ("left_eyebrow", (22, 27)),
    ("right_eye", (36, 42)),
    ("left_eye", (42, 48)),
    ("nose", (27, 35)),
    ("jaw", (0, 17))
])

def eye_aspect_ratio(eye):
    # calcular las distancias euclidianas entre los dos conjuntos de
    # coordenadas verticales del ojo (x, y)
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
 
    # calcular la distancia euclidiana entre la horizontal
    # coordenadas del punto de referencia del ojo (x, y)
    C = dist.euclidean(eye[0], eye[3])
 
    # calcular la proporcion de aspecto del ojo
    ear = (A + B) / (2.0 * C)
    return ear

def resize(img, width=None, height=None, interpolation=cv2.INTER_AREA):
    global ratio
    w, h, _ = img.shape

    if width is None and height is None:
        return img
    elif width is None:
        ratio = height / h
        width = int(w * ratio)
        resized = cv2.resize(img, (height, width), interpolation)
        return resized
    else:
        ratio = width / w
        height = int(h * ratio)
        resized = cv2.resize(img, (height, width), interpolation)
        return resized

def shape_to_np(shape, dtype="int"):
    # inicializar la lista de coordenadas (x, y)
    coords = np.zeros((68, 2), dtype=dtype)

    # bucle sobre los 68 puntos de referencia faciales y convertirlos
    # a una pareja de coordenadas (x, y)
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # devuelve la lista de coordenadas (x, y)
    return coords

camera = cv2.VideoCapture(0)

predictor_path = 'shape_predictor_68_face_landmarks.dat'

print('Cargando el detector de rostros y predictor de puntos de referencia faciales...')
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

# tomar los índices de los puntos de referencia faciales para la izquierda y
# ojo derecho, respectivamente
(lStart, lEnd) = FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = FACIAL_LANDMARKS_IDXS["right_eye"]

vs = cv2.VideoCapture(0)
fileStream = True
time.sleep(1.0)

# bucle sobre los fotogramas del flujo de vídeo
while True:
    # tomar el fotograma del flujo de archivos de vídeo enhebrado, redimensionar
    # y convertirlo en escala de grises
    # canales)
    ret, frame = camera.read()
    if ret == False:
        print('Error al capturar video de la camara. \n')
        break

    frame_resized = resize(frame, width=240)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)

    # detectar rostros en el marco de la escala de grises
    rects = detector(frame_gray, 0)
    # bucle sobre las detecciones de caras
    for rect in rects:
        # determinar los puntos de referencia faciales para la región de la cara, entonces
        # convertir las coordenadas del punto de referencia facial (x, y) en un NumPy
        # arreglo
        shape = predictor(frame_gray, rect)
        shape = shape_to_np(shape)
 
        # extraer las coordenadas del ojo izquierdo y derecho, y luego utilizar el
        # coordenadas para calcular la proporcion de aspecto de ambos ojos
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
 
        # tomar la proporcion de aspecto mínima del ojo
        ear = min([leftEAR,rightEAR])

        # comprobar si la proporcion de aspecto del ojo está por debajo del parpadeo
        # threshold, y si es así, incrementa el contador de fotogramas de parpadeo
        if ear < EYE_AR_THRESH:
            COUNTER += 1

            if 60 >= COUNTER >= EYE_AR_CONSEC_FRAMES:
                cv2.putText(frame, "Cuidado, la persona se esta quedando dormida!", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

            if COUNTER > 80:
                cv2.putText(frame, "La persona esta dormida", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
                
        else:
            # restablecer el contador de fotogramas del ojo
            cv2.putText(frame, "La persona esta despierta", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            COUNTER = 0

        for (x, y) in np.concatenate((leftEye, rightEye), axis=0):
            cv2.circle(frame, (int(x / ratio), int(y / ratio)), 2, (255, 255, 255), -1)

    # mostrar el marco
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
 
    # si se ha pulsado la tecla `q`, romper el bucle
    if key == ord("q"):
        break

cv2.destroyAllWindows()
vs.release()

Cargando el detector de rostros y predictor de puntos de referencia faciales...
